### <span style="color:#22AACC">Pricing European Calls and Puts</span>

In this notebook, we are going to price European vanilla options (calls and puts) with different methods.

- Analytical solution: The Black and Scholes Solution
- Discrete solution: Binomial Tree
- Numerical solution: Monte Carlo Simulation

Let us recall the solution obtained by the B&S equation:

$\begin{equation}
C(S,T) = S_0\Phi(d_1)-e^{-rT}K\Phi(d_2),
\end{equation}$

$\begin{equation}
P(S,T) = e^{-rT}K\Phi(-d_2)-S_0\Phi(-d_1),
\end{equation}$

$\begin{align}
d_1 &= \dfrac{-\ln K+\ln S_0 +(r+\frac{1}{2}\sigma^2)T}{\sigma\sqrt{T}},\nonumber\\
d_2 &= d_1-\sigma\sqrt{T}.
\end{align}$

In [7]:
import numpy as np
from scipy.stats import norm

def vanillaBS(S, K, T, sig, r = 0, call = True):
    d1 = ( np.log(S/K) + (r+sig**2/2)*T ) / (sig*np.sqrt(T))
    d2 = d1 - sig*np.sqrt(T)
    C = S*norm.cdf(d1) - np.exp(-r*T)*K*norm.cdf(d2)
    P = -S*norm.cdf(-d1) + np.exp(-r*T)*K*norm.cdf(-d2)
    return C if call else P


In order to build a Binomial Tree, we assume that the asset can go up by an amount $u$ or down by an amount $d$, where $0 < d < 1 < u $. Furthermore let $f_u$ and $f_d$ be the premium of the option when the asset goes up and down, respectively.

In order for us to have a risk-free portfolio, we want the return to be the same regardless of the asset's direction. Letting $\Delta$ be the amount we have on the underlying asset:


$\begin{equation}
S_0u\Delta-f_u = S_0d\Delta - f_d,
\end{equation}$

or

$\begin{equation}\label{Delta}
\Delta = \dfrac{f_u  - f_d}{S_0u-S_0d}.
\end{equation}$

By the non-arbitrage principle, the cost of building this portfolio must be equal to its final value discounted in time.

Replacing $\Delta$ and isolating $f$:

$\begin{equation}
f = \dfrac{f_u(1-de^{-rT})  - f_d(ue^{-rT}-1)}{u-d}
\end{equation}$

that can be rewritten as

$\begin{equation}
f = e^{-rT}[f_up  - f_d(1-p)],
\end{equation}$

with

$\begin{equation}
p = \dfrac{e^{rT}  - d}{u-d}.
\end{equation}$

In [8]:
def vanillaTree(S, K, T, N, sig, r, d, call=True):
    
    #N is the number of steps, we recall that as N goes to infinity
    #the Binomial Tree converges to the continuous case, i.e., to the B&S solution.
    tree = np.zeros((N+1,N+1))
    dt = T/N
    u = np.exp(sig*np.sqrt(dt))
    d = 1/u
    p = (np.exp(r*dt)-d)/(u-d)
    
    #Building the tree for the underlying asset
    for j in range(N+1):
        for i in range(j+1):
            tree[i,j] = S*u**(j-i)*d**i

    call = np.zeros((N+1,N+1))
    call[:,N] = np.maximum(np.zeros(N+1),tree[:,N]-K)

    #Building the tree for the option
    for j in range(N-1,-1,-1):
        for i in range(j+1):
            call[i,j] = np.exp(-r*dt)*(p*call[i,j+1] + (1-p)*call[i+1,j+1])
            
    return call[0,0]

For the numerical solution, recall that the first assumption is that the price of the asset, $S$, follows a Geometric Brownian Motion:

$\begin{equation}
dS = S\mu dt + S\sigma dX.
\end{equation}$

Applying Itô's lemma to the function $\ln(S)$, one gets:

$\begin{align}
d\ln(S) &= \dfrac{1}{S}(S\mu dt + S\sigma dX)-\dfrac{1}{2}\sigma^2dt,\\
&= (\mu -\dfrac{1}{2}\sigma^2)dt+\sigma dX.
\end{align}$

Thus,

$\begin{equation}
\boxed{S(t) = S(0)e^{(\mu -\frac{1}{2}\sigma^2)t+\sigma (X(t)-X(0))}.}
\end{equation}$

In [9]:
import numpy as np
from scipy.stats import norm

def vanillaMC(S, K, T, N, sig, r):
    np.random.seed(42); dt = 0.1
    W = np.random.randn(N,int(1/dt))*sig*np.sqrt(dt)
    path = np.cumprod(np.exp((r-(sig**2)/2.0)*dt+W),1)*S
    ST = path[:,-1]
    return np.exp(-r*T)*np.maximum(ST-K,0).mean()

In [18]:
#Driver's code
S=100; K=100; T=1; N=200; sig=0.2; r=0.04; d=0; M=5_000_000;

print("Analytical Solution: {:.4}".format(vanillaBS(S,K,T,sig,r,call=True)))
print("Binomial Tree: {:.4}".format(vanillaTree(S,K,T,N,sig,r,d,call=True)))
print("Numerical Solution: {:.4}".format(vanillaMC(S,K,T,M,sig,r)))

Analytical Solution: 9.925
Binomial Tree: 9.915
Numerical Solution: 9.918
